In [3]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [4]:
## new fix for data import
import os
os.environ["HTTP_PROXY"] = ""
os.environ["HTTPS_PROXY"] = ""
os.environ["NO_PROXY"] = ""

In [5]:
## Input Data
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/" + "chase_stacked_final.parquet")
print(chase.shape)

tmobile = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/tmobile_10823/" + "tmbile_file_to_use.parquet")
print(tmobile.shape)

stripe = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/" + "stripe_file_to_use.parquet")
print(stripe.shape)

fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/" + "fleector_sample_to_use.parquet")
print(fleector.shape)

(640958, 36)
(114460, 39)
(75000, 22)
(83704, 92)


Bad pipe message: %s [b'\x83Cij\xa1O~', b"ns\x17\x9e\xe9\xf1\xbf\xfd\xa2 \x00\xcdocW\x04\xd4\xfd\xa1\xf4\xc7\x08\xd2\x81y\x98Y&\xaev\x18n\xd3\x02\xe1\xc7=\x1a'\xaa9\xb0\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xf3\x87\xa6I\xeer\x13\xf7T[MQmo\xc8%\x04\xc2U\xb0\xf9\x9dkZ\x97\x94]@)\xd8\xdc\x06"]
Bad pipe message: %s [b'\xb6\x1b\xdc\xeafs\x84\x1f\x92\xda\x10\n\xe4\xd1-F\xc7\xca KT"W\xec\x99n\xc9\x00v\x98Iz&\xcc\xe1\xd0\x19po9\xdb\xd6\xfb\xb76\xbcP\x15\x03\x1a\xc4\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00']
Bad pipe message: %s [b'\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x0

In [3]:
## keep seleid, full address, phone, taxid, performance flag >> business information >> might need to dedup

#### SBFE Inquiry

In [4]:
path = "Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Data Modeling Clean/sbfe_seleid_append_deduped_cleaned_0501.parquet"
sbfe = pd.read_adls(path, reader = pd.read_parquet)
print(sbfe.shape)

(79429086, 20)


In [5]:
sbfe["full_addr"] = sbfe["addr"] + np.where(sbfe["city"] != "", ", " + sbfe["city"], "") + \
    np.where(sbfe["state"] != "", ", " + sbfe["state"], "") + np.where(sbfe["zip"] != "", ", " + sbfe["zip"].str.slice(0, 5), "")
sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('.', '')
sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace(',', '')
sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace('\W\s', '')
sbfe["full_addr_clean"] = sbfe["full_addr_clean"].str.lower()

/tmp/ipykernel_2383/3858317675.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sbfe["full_addr_clean"] = sbfe['full_addr'].str.replace('.', '')
/tmp/ipykernel_2383/3858317675.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  sbfe["full_addr_clean"] = sbfe['full_addr_clean'].str.replace('\W\s', '')


In [6]:
sbfe = sbfe[["seleid", "historydate_x", "full_addr_clean"]]
sbfe.columns = ["seleid_inquiry_sbfe", "inquiry_date", "full_addr_clean"]

In [7]:
sbfe["inquiry_date"] = sbfe["inquiry_date"].astype(int)
sbfe["inquiry_date_fmt"] = pd.to_datetime(sbfe["inquiry_date"], format='%Y%m%d')

In [8]:
sbfe["seleid_inquiry_sbfe"] = sbfe["seleid_inquiry_sbfe"].astype(float)

In [9]:
sbfe["full_addr_clean"] = sbfe["full_addr_clean"].str.lower()

In [10]:
sbfe.head()

,seleid_inquiry_sbfe,inquiry_date,full_addr_clean,inquiry_date_fmt
0,1.336156e+11,20210701,115 golf course rd ste e logan ut 84321,2021-07-01
1,1.330122e+09,20210701,3200 palm tree dr lithonia ga 30038,2021-07-01
2,9.064136e+08,20210701,1476 highway 159 e bellville tx 77418,2021-07-01
5,6.136772e+07,20210701,492 koller st san francisco ca 94110,2021-07-01
6,1.371452e+11,20210701,107 lassiter lane bellville tx 77418,2021-07-01


#### Stripe 11363

In [11]:
stripe = stripe.fillna("")
stripe = stripe.drop_duplicates(subset = ['companyname', 'alternatecompanyname', 'bus_streetaddress1', 
                                          'bus_streetaddress2', 'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website'], ignore_index = True) 
print(stripe.shape)
print(stripe.uniqueid.is_unique)

(74443, 22)
True


In [12]:
# Compile SELEIDS
# stripe_sba = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/stripe_11363/final_str_11363_20221025file_sba21_nonsbfe.csv.gz",
#                       compression = "gzip", reader = pd.read_csv)
# print(stripe_sba.shape)
# stripe_sba = stripe_sba.drop_duplicates(subset = ["uniqueid"], ignore_index = True) 
# print(stripe_sba.shape)
# keep = ['uniqueid', 'lnlexidsele',  'inquirycount03m', 'inquirycount12m', 'inquirycreditcount03m', 'inquirycreditcount12m',]
# stripe_sba = stripe_sba[keep]
# stripe = stripe.merge(stripe_sba, on = "uniqueid")
# print(stripe.shape)

In [13]:
stripe.columns

Index(['Unnamed: 0', 'uniqueid', 'historydate', 'companyname',
       'alternatecompanyname', 'bus_streetaddress1', 'bus_streetaddress2',
       'bus_city', 'bus_state', 'bus_zip', 'bus_phone', 'bus_website',
       'firstname', 'lastname', 'streetaddress1', 'streetaddress2', 'city',
       'state', 'zip', 'email', 'ip_address', 'performance_indicator'],
      dtype='object')

In [14]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
stripe["full_addr"] = stripe["bus_streetaddress1"] + \
    np.where(stripe["bus_city"] != "", ", " + stripe["bus_city"], "") + \
    np.where(stripe["bus_state"] != "", ", " + stripe["bus_state"], "") + np.where(stripe["bus_zip"] != "", ", " + stripe["bus_zip"].str.slice(0, 5), "")
stripe["full_addr_clean"] = stripe['full_addr'].str.replace('.', '')
stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace(',', '')
stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace('\W\s', '')
stripe["full_addr_clean"] = stripe["full_addr_clean"].str.lower()

/tmp/ipykernel_2383/2995735239.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stripe["full_addr_clean"] = stripe['full_addr'].str.replace('.', '')
/tmp/ipykernel_2383/2995735239.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  stripe["full_addr_clean"] = stripe['full_addr_clean'].str.replace('\W\s', '')


In [15]:
stripe[["full_addr_clean", "full_addr"]] 

,full_addr_clean,full_addr
0,2093 philadelphia pike claymont de 19703,"2093 Philadelphia Pike, Claymont, DE, 19703"
1,11415 segrell way culver city ca 90230,"11415 Segrell Way, Culver City, CA, 90230"
2,361 friendsville road markleysburg pa 15459,"361 Friendsville Road, Markleysburg, PA, 15459"
3,237 ne floresta drive port saint luciefl 34983,"237 NE Floresta Drive, Port Saint Lucie , FL, ..."
4,611 broadway new york ny 10012,"611 Broadway, New York, NY, 10012"
...,...,...
74438,949 valley road blue bell pa 19422,"949 Valley Road, Blue Bell, PA, 19422"
74439,189 oak valley rd mcdonough ga 30253,"189 Oak Valley Rd, Mcdonough, GA, 30253"
74440,1910 thomes avenue cheyenne wy 82001,"1910 Thomes Avenue, Cheyenne, WY, 82001"
74441,62 greene st 2nd fl new york ny 10012,"62 Greene St., 2nd Fl., New York, NY, 10012"


In [16]:
freq(stripe.historydate.str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
historydate,,,,
2022-01,24426.0,0.328117,24426.0,0.328117
2022-02,24209.0,0.325202,48635.0,0.653319
2022-03,25808.0,0.346681,74443.0,1.000000


In [17]:
freq(stripe.performance_indicator)

,Count,Pct,Cuml Count,Cuml Pct
performance_indicator,,,,
False,73807.0,0.991457,73807.0,0.991457
True,636.0,0.008543,74443.0,1.000000


In [18]:
stripe["bad"] = np.where(stripe.performance_indicator == "True", 1, 0)

In [20]:
stripe[stripe["full_addr_clean"]!=""].shape

(74438, 25)

In [21]:
stripe = stripe[stripe["full_addr_clean"]!=""]
print(stripe.shape)

(74438, 25)


In [22]:
stripe["credit_flag_cutoff_date"] = stripe["historydate"].str.slice(0, 10)
stripe["credit_flag_cutoff_date_fmt"] = pd.to_datetime(stripe["credit_flag_cutoff_date"], format='%Y-%m-%d')

/tmp/ipykernel_2383/3608108231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stripe["credit_flag_cutoff_date"] = stripe["historydate"].str.slice(0, 10)
/tmp/ipykernel_2383/3608108231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stripe["credit_flag_cutoff_date_fmt"] = pd.to_datetime(stripe["credit_flag_cutoff_date"], format='%Y-%m-%d')


In [23]:
sbfe_for_stripe = sbfe.merge(stripe, on = "full_addr_clean")
print(sbfe_for_stripe.shape)
sbfe_for_stripe = sbfe_for_stripe[sbfe_for_stripe["inquiry_date_fmt"] < sbfe_for_stripe["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_stripe.shape)

sbfe_for_stripe['inquiry_count_06']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_stripe['inquiry_count_03']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_stripe['inquiry_count_01']   = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_stripe['inquiry_count_week'] = np.where((sbfe_for_stripe["credit_flag_cutoff_date_fmt"] - sbfe_for_stripe["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_stripe_roll = sbfe_for_stripe.groupby(by = ['uniqueid'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_stripe_roll.shape)

(410374, 30)
(250294, 30)
(10557, 5)


/tmp/ipykernel_2383/3242491595.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_stripe_roll = sbfe_for_stripe.groupby(by = ['uniqueid'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [24]:
stripe = stripe.merge(sbfe_for_stripe_roll, on = "uniqueid", how = "left")
print(stripe.shape)

(74438, 31)


In [25]:
stripe['inquiry_count_06'] = np.where(stripe["inquiry_count_06"].isnull(), 0,   stripe["inquiry_count_06"])
stripe['inquiry_count_03'] = np.where(stripe["inquiry_count_03"].isnull(), 0,   stripe["inquiry_count_03"])
stripe['inquiry_count_01'] = np.where(stripe["inquiry_count_01"].isnull(), 0,   stripe["inquiry_count_01"])
stripe['inquiry_count_week']=np.where(stripe["inquiry_count_week"].isnull(), 0, stripe["inquiry_count_week"])

In [26]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ["inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(stripe[a], stripe["bad"], format = fmt)

wb = TableWriter(filename = "./_temp/stripe.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### TMobile 10823

In [27]:
tmobile["credit_flag_cutoff_date"] = tmobile.appdatetime_gmt.str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [28]:
freq(tmobile["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,20510.0,0.179189,20510.0,0.179189
2022-02,22452.0,0.196156,42962.0,0.375345
2022-03,22846.0,0.199598,65808.0,0.574943
2022-04,14973.0,0.130814,80781.0,0.705757
2022-05,16271.0,0.142154,97052.0,0.847912
2022-06,17408.0,0.152088,114460.0,1.000000


In [29]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,33679.0,0.294243,33679.0,0.294243
retro,80781.0,0.705757,114460.0,1.000000


In [30]:
freq(tmobile.fraud_ind)

,Count,Pct,Cuml Count,Cuml Pct
fraud_ind,,,,
0,58655.0,0.512450,58655.0,0.512450
1,3762.0,0.032867,62417.0,0.545317
Missing,52043.0,0.454683,114460.0,1.000000


In [31]:
tmobile = tmobile[tmobile["fraud_ind"].notnull()]
print(tmobile.shape)

(62417, 41)


In [32]:
freq(tmobile.src)

,Count,Pct,Cuml Count,Cuml Pct
src,,,,
march,21914.0,0.35109,21914.0,0.35109
retro,40503.0,0.64891,62417.0,1.00000


In [33]:
tmobile["credit_flag_cutoff_date"] = tmobile["appdatetime_gmt"].str.slice(0, 9)
tmobile["credit_flag_cutoff_date_fmt"] = pd.to_datetime(tmobile["credit_flag_cutoff_date"], format='%d%b%Y')

In [34]:
tmobile.columns

Index(['crid_encr', 'appdatetime_gmt', 'businessname', 'business_contactphone',
       'business_addressline1', 'business_addressline2',
       'business_addresscity', 'business_addressstate',
       'business_addresspostalcode', 'federaltaxid', 'contactfirstname',
       'contactmiddlename', 'contactlastname', 'contact_contactphone',
       'contact_addressline1', 'contact_addressline2', 'contact_addresscity',
       'contact_addressstate', 'contact_addresspostalcode', 'contact_dob',
       'contact_idnum', 'contact_idtype', 'contact_idstate', 'contact_email',
       'application_channel', 'fraud_ind', 'fraud_tp_ind', 'fraud_fp_ind',
       'fpd_ind', 'any_wo_ind', 'activationflag', 'seq', 'account', 'date',
       'src', 'contact_dob_year', 'contact_dob_month', 'contact_dob_day',
       'contact_driverlicensenumber', 'credit_flag_cutoff_date',
       'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [37]:
tmobile['abbrev_state'] = tmobile['business_addressstate'].map(statename_to_abbr)
print(tmobile['abbrev_state'].isnull().sum())
print(tmobile['business_addressstate'].isnull().sum())

3
3


In [83]:
# freq("business_addressstate", df=tmobile[tmobile['business_addressstate'].notnull() & tmobile['abbrev_state'].isnull()])
## ADD abbreviation to mapping dictionary

,Count,Pct,Cuml Count,Cuml Pct
business_addressstate,,,,
Puerto Rico,268.0,0.985294,268.0,0.985294
Virgin Islands,4.0,0.014706,272.0,1.000000


In [38]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
tmobile["full_addr"] = tmobile["business_addressline1"] + \
    np.where(tmobile["business_addresscity"] != "", ", " + tmobile["business_addresscity"], "") + \
    np.where(tmobile["abbrev_state"] != "", ", " + tmobile["abbrev_state"], "") + \
        np.where(tmobile["business_addresspostalcode"] != "", ", " + tmobile["business_addresspostalcode"].str.slice(0, 5), "")
tmobile["full_addr_clean"] = tmobile['full_addr'].str.replace('.', '')
tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace(',', '')
tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace('\W\s', '')
tmobile["full_addr_clean"] = tmobile["full_addr_clean"].str.lower()

/tmp/ipykernel_2383/3611770108.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tmobile["full_addr_clean"] = tmobile['full_addr'].str.replace('.', '')
/tmp/ipykernel_2383/3611770108.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tmobile["full_addr_clean"] = tmobile['full_addr_clean'].str.replace('\W\s', '')


In [42]:
tmobile[tmobile["full_addr_clean"].notnull()].shape

(62414, 44)

In [43]:
tmobile = tmobile[tmobile["full_addr_clean"].notnull()]

In [44]:
sbfe_for_tmobile = sbfe.merge(tmobile, on = "full_addr_clean")
print(sbfe_for_tmobile.shape)
sbfe_for_tmobile = sbfe_for_tmobile[sbfe_for_tmobile["inquiry_date_fmt"] < sbfe_for_tmobile["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_tmobile.shape)

sbfe_for_tmobile['inquiry_count_06']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_tmobile['inquiry_count_03']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_tmobile['inquiry_count_01']   = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_tmobile['inquiry_count_week'] = np.where((sbfe_for_tmobile["credit_flag_cutoff_date_fmt"] - sbfe_for_tmobile["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_tmobile_roll = sbfe_for_tmobile.groupby(by = ['crid_encr'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_tmobile_roll.shape)

(274727, 47)
(201828, 47)
(14598, 5)


/tmp/ipykernel_2383/3772620228.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_tmobile_roll = sbfe_for_tmobile.groupby(by = ['crid_encr'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [45]:
tmobile = tmobile.merge(sbfe_for_tmobile_roll, on = "crid_encr", how = "left")
print(tmobile.shape)

(62414, 48)


In [46]:
tmobile['inquiry_count_06'] = np.where(tmobile["inquiry_count_06"].isnull(), 0,   tmobile["inquiry_count_06"])
tmobile['inquiry_count_03'] = np.where(tmobile["inquiry_count_03"].isnull(), 0,   tmobile["inquiry_count_03"])
tmobile['inquiry_count_01'] = np.where(tmobile["inquiry_count_01"].isnull(), 0,   tmobile["inquiry_count_01"])
tmobile['inquiry_count_week']=np.where(tmobile["inquiry_count_week"].isnull(), 0, tmobile["inquiry_count_week"])

In [47]:
tmobile["fraud_ind"] = tmobile["fraud_ind"].astype(float)

In [48]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ["inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(tmobile[a], tmobile["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/tmobile.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Chase

In [49]:
chase = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/Chase_11449/chase_stacked_final.parquet")
print(chase.shape)

(640958, 36)


In [50]:
chase["credit_flag_cutoff_date_fmt"] = pd.to_datetime(chase["app_date"], format='%d%b%Y')

In [51]:
chase.columns

Index(['transaction_id', 'cust_first_nm', 'cust_mid_init_tx', 'cust_last_nm',
       'govt_issu_id_nb', 'line_1_ad', 'line_2_ad', 'city_nm', 'state_prov_cd',
       'pst_area_7_cd', 'pst_area_cd', 'channel', 'bus_name', 'app_date',
       'date_of_birth', 'decision', 'bizidscore', 'bd_score', 'bd_score1',
       'bd_score2', 'bd_score3', 'bus_addr1', 'bus_addr2', 'bus_city',
       'bus_state', 'bus_zip', 'bus_tax_id', 'bus_phone', 'bus_type',
       'home_phone', 'bad', 'seq', 'account', 'date', 'LN_booked', 'count',
       'credit_flag_cutoff_date_fmt'],
      dtype='object')

In [52]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
chase["full_addr"] = chase["bus_addr1"] + \
    np.where(chase["bus_city"] != "", ", " + chase["bus_city"], "") + \
    np.where(chase["bus_state"] != "", ", " + chase["bus_state"], "") + \
        np.where(chase["bus_zip"] != "", ", " + chase["bus_zip"].str.slice(0, 5), "")
chase["full_addr_clean"] = chase['full_addr'].str.replace('.', '')
chase["full_addr_clean"] = chase['full_addr_clean'].str.replace(',', '')
chase["full_addr_clean"] = chase['full_addr_clean'].str.replace('\W\s', '')
chase["full_addr_clean"] = chase["full_addr_clean"].str.lower()
chase["full_addr_clean"]

/tmp/ipykernel_2383/3843543105.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chase["full_addr_clean"] = chase['full_addr'].str.replace('.', '')
/tmp/ipykernel_2383/3843543105.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  chase["full_addr_clean"] = chase['full_addr_clean'].str.replace('\W\s', '')


499427                 2869 sw 33rd ct unit 1 miami fl 33133
837612              821 michigan ave apt 3 evanston il 60202
1018748             8204 n lamar blvd ste a4 austin tx 78753
82631                      20558 noland woods ct sterling va
37307                              23 n midland ave nyack ny
                                 ...                        
156867                 2222 s dobson rd apt 1089 chandler az
140946                          1018 n morrison st kokomo in
62551                  597 westport ave unit a420 norwalk ct
1225640    17070 collins ave ste 255 sunny isles beach fl...
49725                 8324 seven falls st north las vegas nv
Name: full_addr_clean, Length: 640958, dtype: object

In [53]:
print(chase[chase["full_addr_clean"]==""].shape)
print(chase[chase["full_addr_clean"].isnull()].shape)

(0, 39)
(0, 39)


In [97]:
sbfe_for_chase = sbfe.merge(chase, on = "full_addr_clean")
print(sbfe_for_chase.shape)
sbfe_for_chase = sbfe_for_chase[sbfe_for_chase["inquiry_date_fmt"] < sbfe_for_chase["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_chase.shape)

sbfe_for_chase['inquiry_count_06']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_chase['inquiry_count_03']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_chase['inquiry_count_01']   = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_chase['inquiry_count_week'] = np.where((sbfe_for_chase["credit_flag_cutoff_date_fmt"] - sbfe_for_chase["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_chase_roll = sbfe_for_chase.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_chase_roll.shape)

(2658103, 42)
(1921750, 42)


/tmp/ipykernel_5854/4000701633.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_chase_roll = sbfe_for_chase.groupby(by = ['account'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


(95130, 5)


In [98]:
chase = chase.merge(sbfe_for_chase_roll, on = "account", how = "left")
print(chase.shape)

(640958, 43)


In [99]:
chase['inquiry_count_06'] = np.where(chase["inquiry_count_06"].isnull(), 0,   chase["inquiry_count_06"])
chase['inquiry_count_03'] = np.where(chase["inquiry_count_03"].isnull(), 0,   chase["inquiry_count_03"])
chase['inquiry_count_01'] = np.where(chase["inquiry_count_01"].isnull(), 0,   chase["inquiry_count_01"])
chase['inquiry_count_week']=np.where(chase["inquiry_count_week"].isnull(), 0, chase["inquiry_count_week"])

In [100]:
chase["fraud_ind"] = chase["bad"].astype(float)

In [101]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ["inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(chase[a].astype(float), chase["fraud_ind"], format = fmt)

wb = TableWriter(filename = "./_temp/chase.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Fleector

In [54]:
fleector = pd.read_adls("Analytics/RnD Projects/Product RnD/Business/LexisNexis 9999 (SBFE Inquiry POC)/Performance/Fraud Sample/fleector_11124/fleector_sample_to_use.parquet")
print(fleector.shape)

(83704, 92)


In [55]:
fleector["credit_flag_cutoff_date_fmt"] = pd.to_datetime(fleector["date_created"], format='%Y-%m-%d')

In [56]:
freq(fleector["credit_flag_cutoff_date_fmt"].astype(str).str.slice(0, 7))

,Count,Pct,Cuml Count,Cuml Pct
credit_flag_cutoff_date_fmt,,,,
2022-01,15827.0,0.189083,15827.0,0.189083
2022-02,15512.0,0.185320,31339.0,0.374403
2022-03,19353.0,0.231208,50692.0,0.605610
2022-04,16822.0,0.200970,67514.0,0.806580
2022-05,16185.0,0.193360,83699.0,0.999940
2022-06,5.0,0.000060,83704.0,1.000000


In [57]:
fleector.columns

Index(['seq', 'ats_id', 'open_date', 'open_mon', 'open_qtr',
       'writeoff_flag_12mo', 'writeoff_flag_18mo', 'wo_amount', 'wo_date',
       'wo_mon', 'wo_qtr', 'pmt_sum_12mo', 'pmt_count_12mo',
       'return_pmt_count_12mo', 'pmt_sum_18mo', 'pmt_count_18mo',
       'return_pmt_count_18mo', 'platform', 'lock_code', 'lock_reason',
       'revenue_12mo', 'revenue_18mo', 'fuel_revenue_12mo',
       'fuel_revenue_18mo', 'fee_revenue_12mo', 'fee_revenue_18mo',
       'account_code', 'limit_approved', 'last_pos_cred_limit', 'gallons_12mo',
       'gallons_18mo', 'roadster_twentile', 'ats_bill_cycle_group', 'app_flag',
       'acct_flag', 'app_mon', 'date_created', 'channel', 'team',
       'processor_cd', 'brand', 'product', 'portfolio', 'status_credit',
       'status_fraud', 'creditstatus', 'req_credit_limit', 'builder_pro_flag',
       'test_app', 'dup_match', 'approve_flag', 'cond_deposit_category',
       'secured_approval_flag', 'cust_name', 'fed_id', 'nmf_contact_email',
       'bu

In [60]:
# Compile Full Address: np.where(stripe["bus_streetaddress2"] != "", ", " + stripe["bus_streetaddress2"], "") + 
fleector["full_addr"] = fleector["cust_address1"] + \
    np.where(fleector["cust_city"] != "", ", " + fleector["cust_city"], "") + \
    np.where(fleector["cust_state"] != "", ", " + fleector["cust_state"], "") + \
        np.where(fleector["cust_zip"] != "", ", " + fleector["cust_zip"].str.slice(0, 5), "")
fleector["full_addr_clean"] = fleector['full_addr'].str.replace('.', '')
fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace(',', '')
fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace('\W\s', '')
fleector["full_addr_clean"] = fleector["full_addr_clean"].str.lower()
fleector["full_addr_clean"]

/tmp/ipykernel_2383/2826320773.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  fleector["full_addr_clean"] = fleector['full_addr'].str.replace('.', '')
/tmp/ipykernel_2383/2826320773.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  fleector["full_addr_clean"] = fleector['full_addr_clean'].str.replace('\W\s', '')


6875                  637 sackman st brooklyn ny 11212
5628     413timberline ridge ct winston salem nc 27106
19723          170bentley place way covington ga 30016
4981                99 horace lane walterboro sc 29488
17116           3828 highlands pkwy se smyrna ga 30082
                             ...                      
49448           10387 tiago lake dr escondido ca 92029
705          645 henderson drive cartersville ga 30120
79073             98 whitethorne ave columbus oh 43223
61964               107 floridana road debary fl 32713
54909           4207 youngstown dr greensboro nc 27405
Name: full_addr_clean, Length: 83704, dtype: object

In [61]:
print(fleector[fleector["full_addr_clean"]==""].shape)
print(fleector[fleector["full_addr_clean"].isnull()].shape)

(0, 95)
(0, 95)


In [110]:
fleector.seq.is_unique

True

In [111]:
sbfe_for_fleector = sbfe.merge(fleector, on = "full_addr_clean")
print(sbfe_for_fleector.shape)
sbfe_for_fleector = sbfe_for_fleector[sbfe_for_fleector["inquiry_date_fmt"] < sbfe_for_fleector["credit_flag_cutoff_date_fmt"]]
print(sbfe_for_fleector.shape)

sbfe_for_fleector['inquiry_count_06']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 6, 1, 0)
sbfe_for_fleector['inquiry_count_03']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 3, 1, 0)
sbfe_for_fleector['inquiry_count_01']   = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'M') <= 1, 1, 0)
sbfe_for_fleector['inquiry_count_week'] = np.where((sbfe_for_fleector["credit_flag_cutoff_date_fmt"] - sbfe_for_fleector["inquiry_date_fmt"])/np.timedelta64(1, 'W') <= 1, 1, 0)

sbfe_for_fleector_roll = sbfe_for_fleector.groupby(by = ['seq'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()
print(sbfe_for_fleector_roll.shape)

(176804, 98)
(89741, 98)
(14482, 5)


/tmp/ipykernel_5854/2289841132.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sbfe_for_fleector_roll = sbfe_for_fleector.groupby(by = ['seq'])["inquiry_count_06", "inquiry_count_03", "inquiry_count_01", "inquiry_count_week"].sum().reset_index()


In [112]:
fleector = fleector.merge(sbfe_for_fleector_roll, on = "seq", how = "left")
print(fleector.shape)

(83704, 99)


In [113]:
fleector['inquiry_count_06'] = np.where(fleector["inquiry_count_06"].isnull(), 0,   fleector["inquiry_count_06"])
fleector['inquiry_count_03'] = np.where(fleector["inquiry_count_03"].isnull(), 0,   fleector["inquiry_count_03"])
fleector['inquiry_count_01'] = np.where(fleector["inquiry_count_01"].isnull(), 0,   fleector["inquiry_count_01"])
fleector['inquiry_count_week']=np.where(fleector["inquiry_count_week"].isnull(), 0, fleector["inquiry_count_week"])

In [114]:
fleector["fraud_ind"] = fleector["final_fraud_flag"].astype(float)

In [115]:
fmt = make_format(cuts = [-np.inf, 0, 1, 2, 3, 4, 5, 10, 20, 30, 40, 50, np.inf], exceptions = [-1])
result = dict()
keep = ["inquiry_count_06", 'inquiry_count_03', 'inquiry_count_01', 'inquiry_count_week']
for i, a in enumerate(keep):
    result[i] = bivariate(fleector[a].astype(float), fleector["fraud_ind"], format = fmt, groups= fleector.acct_flag)

wb = TableWriter(filename = "./_temp/fleector.xlsx", options={'nan_inf_to_errors': True}, overwrite = True)
for k in result.keys():
    wb.write_table(
        result[k],
        sheetname="biv",
        conditional_fmt_cols=[3],
    )
wb.close()

#### Other

In [36]:
statename_to_abbr = {
    # Other
    'District of Columbia': 'DC',

    # States
    'Alabama': 'AL',
    'Montana': 'MT',
    'Alaska': 'AK',
    'Nebraska': 'NE',
    'Arizona': 'AZ',
    'Nevada': 'NV',
    'Arkansas': 'AR',
    'New Hampshire': 'NH',
    'California': 'CA',
    'New Jersey': 'NJ',
    'Colorado': 'CO',
    'New Mexico': 'NM',
    'Connecticut': 'CT',
    'New York': 'NY',
    'Delaware': 'DE',
    'North Carolina': 'NC',
    'Florida': 'FL',
    'North Dakota': 'ND',
    'Georgia': 'GA',
    'Ohio': 'OH',
    'Hawaii': 'HI',
    'Oklahoma': 'OK',
    'Idaho': 'ID',
    'Oregon': 'OR',
    'Illinois': 'IL',
    'Pennsylvania': 'PA',
    'Indiana': 'IN',
    'Rhode Island': 'RI',
    'Iowa': 'IA',
    'South Carolina': 'SC',
    'Kansas': 'KS',
    'South Dakota': 'SD',
    'Kentucky': 'KY',
    'Tennessee': 'TN',
    'Louisiana': 'LA',
    'Texas': 'TX',
    'Maine': 'ME',
    'Utah': 'UT',
    'Maryland': 'MD',
    'Vermont': 'VT',
    'Massachusetts': 'MA',
    'Virginia': 'VA',
    'Michigan': 'MI',
    'Washington': 'WA',
    'Minnesota': 'MN',
    'West Virginia': 'WV',
    'Mississippi': 'MS',
    'Wisconsin': 'WI',
    'Missouri': 'MO',
    'Wyoming': 'WY',
    "Puerto Rico" : "PR",
"Virgin Islands" : "VI"
}